In [2]:
# Y-Variable #

import numpy as np
import pandas as pd
import yfinance as yf
import warnings

warnings.filterwarnings("ignore")

symbols = pd.read_csv("stock_symbols_new.csv")

rand_symbols = symbols['symbol'].head(50).tolist()
print(rand_symbols)

def forward_excess_return_monthly(tickers, benchmark="SPY", start="2010-01-01", end=None, horizon_months=3):

    universe = list(dict.fromkeys(list(tickers) + [benchmark]))

    px_daily = yf.download(
        universe, start=start, end=end, auto_adjust=False, progress=False
    )["Adj Close"]

    px_m = px_daily.resample("M").last().dropna(subset=[benchmark])

    fwd_ret = px_m.shift(-horizon_months) / px_m - 1.0
    bench_fwd = fwd_ret[benchmark].rename("bench_fwd_return")

    # Wide to long in one go
    out = pd.DataFrame(index=px_m.index)
    out["bench_fwd_return"] = bench_fwd

    for t in tickers:
        out[f"adj_close_{t}"] = px_m[t]
        out[f"fwd_return_{t}"] = fwd_ret[t]
        out[f"fwd_excess_{t}"] = fwd_ret[t] - bench_fwd

    return out

df_y_m = forward_excess_return_monthly(rand_symbols, benchmark="SPY", start="2010-01-01", horizon_months=6)
print(df_y_m.head(10))
print(df_y_m.tail(10))

bench_df = (
    df_y_m[["bench_fwd_return"]]
    .rename(columns={"bench_fwd_return": "bench_fwd_return"})
    .reset_index()
)

# Keep only stock-level columns
stock_cols = [c for c in df_y_m.columns if "_" in c and not c.startswith("fwd_ret_bench")]

y_long = (
    df_y_m[stock_cols]
    .reset_index()
    .melt(id_vars="Date", var_name="metric_ticker", value_name="value")
)

# Split "adj_close_AAPL" -> metric="adj_close", ticker="AAPL"
y_long[["metric", "ticker"]] = y_long["metric_ticker"].str.rsplit("_", n=1, expand=True)

y_long = y_long.drop(columns="metric_ticker")

df_y_m_output = (
    y_long
    .pivot(index=["Date", "ticker"], columns="metric", values="value")
    .reset_index()
)

df_y_m_output = df_y_m_output[["Date", "ticker", "adj_close", "fwd_excess", "fwd_return"]]
df_y_m_output = df_y_m_output[df_y_m_output["ticker"] != "return"]

print(df_y_m_output.head(10))
print(df_y_m_output.tail(10))


# Save to CSV
df_y_m_output.to_csv("y_variable_forward_excess_return.csv", index=False)



['NVDA', 'GOOGL', 'AAPL', 'MSFT', 'AMZN', 'META', 'AVGO', 'TSLA', 'BRK-B', 'LLY', 'WMT', 'JPM', 'V', 'SPY', 'ORCL', 'XOM', 'MA', 'ASML', 'JNJ', 'PLTR', 'BAC', 'COST', 'ABBV', 'MU', 'NFLX', 'HD', 'BABA', 'GE', 'PG', 'AMD', 'CVX', 'UNH', 'WFC', 'KO', 'MS', 'AZN', 'CYATY', 'CSCO', 'CAT', 'TM', 'SAP', 'IBM', 'GS', 'MRK', 'LRCX', 'HSBC', 'NVS', 'AXP', 'NVO', 'PM']
            bench_fwd_return  adj_close_NVDA  fwd_return_NVDA  \
Date                                                            
2010-01-31          0.035952        0.352752        -0.402859   
2010-02-28         -0.040574        0.371318        -0.424074   
2010-03-31         -0.014642        0.398823        -0.328735   
2010-04-30          0.007416        0.360087        -0.234882   
2010-05-31          0.094369        0.301180         0.035769   
2010-06-30          0.231235        0.234022         0.508325   
2010-07-31          0.179371        0.210643         1.602830   
2010-08-31          0.277816        0.213852         